In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [76]:
os.environ["OPENROUTER_API_KEY"] = "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"

In [77]:
os.environ["OPENROUTER_API_KEY"]

'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'

In [156]:
import requests
url = "https://openrouter.ai/api/v1/models"
response = requests.get(url)
d = response.json()['data']
[i['id'] for i in d if i['id'].startswith('google')]

['google/gemini-2.5-flash-preview-09-2025',
 'google/gemini-2.5-flash-lite-preview-09-2025',
 'google/gemini-2.5-flash-image-preview',
 'google/gemini-2.5-flash-lite',
 'google/gemma-3n-e2b-it:free',
 'google/gemini-2.5-flash-lite-preview-06-17',
 'google/gemini-2.5-flash',
 'google/gemini-2.5-pro',
 'google/gemini-2.5-pro-preview',
 'google/gemma-3n-e4b-it:free',
 'google/gemma-3n-e4b-it',
 'google/gemini-2.5-pro-preview-05-06',
 'google/gemma-3-4b-it:free',
 'google/gemma-3-4b-it',
 'google/gemma-3-12b-it:free',
 'google/gemma-3-12b-it',
 'google/gemma-3-27b-it:free',
 'google/gemma-3-27b-it',
 'google/gemini-2.0-flash-lite-001',
 'google/gemini-2.0-flash-001',
 'google/gemini-2.0-flash-exp:free',
 'google/gemma-2-27b-it',
 'google/gemma-2-9b-it:free',
 'google/gemma-2-9b-it']

In [157]:
import os
import json
import re
import pandas as pd
from typing import Set, List
from copy import deepcopy
from min_tom import *
from eval import *

In [178]:
models=[
    'anthropic/claude-sonnet-4', 
    'anthropic/claude-sonnet-4.5', 
    'anthropic/claude-opus-4.1', 
    # 'openai/gpt-5',
    # 'google/gemini-2.5-pro',
]
agents=['Sally','Anne','Emma',]# 'Benjamin', 'Aiden', 'Isabella']
rooms = {
    'kitchen': {'objects': ['ball'], 'containers': ['cabinet','drawer']},
    'room': {'objects': ['marble', 'ball'], 'containers': ['basket','box', 'cabinet']},
}
def engine_factory(story_id):
    return Engine(agents=agents, rooms=rooms, seed=story_id + 100,
        peek_prob=0., distracted_prob=0., exit_without_move_prob=0.2, allow_other_actions=True)
# story, results = run_evaluation(engine, steps=10, models=models, sleep=0.5*0, max_workers=10)
df = run_evaluations(engine_factory, num_stories=1, steps=10, models=models, max_workers=10)


break at step 31/40
Story 0: 42 QA items, 0 peeks, 0 distractions.
Evaluating 42 QAs with model anthropic/claude-sonnet-4 (max_workers=10)...


Evaluating anthropic/claude-sonnet-4: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:12<00:00,  3.29QA/s]


Evaluating 42 QAs with model anthropic/claude-sonnet-4.5 (max_workers=10)...


Evaluating anthropic/claude-sonnet-4.5: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:13<00:00,  3.16QA/s]


Evaluating 42 QAs with model anthropic/claude-opus-4.1 (max_workers=10)...


Evaluating anthropic/claude-opus-4.1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:18<00:00,  2.33QA/s]


In [ ]:
df.groupby(["model", "qa_order"]).is_correct.mean()  # non-reasoning models

model                        qa_order
anthropic/claude-opus-4.1    1           0.807692
                             2           0.692308
anthropic/claude-sonnet-4    1           0.807692
                             2           0.538462
anthropic/claude-sonnet-4.5  1           0.730769
                             2           0.641026
Name: is_correct, dtype: float64

In [153]:
(
    df.assign(story_question=df["story_id"].astype(str) + " | " + df["question"])
      .groupby("story_question", as_index=False)
      .agg(
          is_correct_mean=("is_correct", "mean"),
          qa_order=("qa_order", "first"),
          n_peeks=("n_peeks", "first"),
          n_distractions=("n_distractions", "first"),
      )
      .sort_values("is_correct_mean")
      .head(20)
)

,story_question,is_correct_mean,qa_order,n_peeks,n_distractions
54,8 | Where does Sally think Emma thinks the mar...,0.000000,2,3,1
62,9 | Where does Sally think Anne thinks the mar...,0.000000,2,2,2
36,5 | Where does Sally think the marble is?,0.000000,1,1,0
35,5 | Where does Sally think Anne thinks the mar...,0.000000,2,1,0
56,9 | Where does Anne think Emma thinks the marb...,0.000000,2,2,2
63,9 | Where does Sally think Emma thinks the mar...,0.000000,2,2,2
6,1 | Where does Anne think the marble is?,0.000000,1,1,1
30,5 | Where does Anne think Sally thinks the mar...,0.000000,2,1,0
17,2 | Where does Emma think the marble is?,0.000000,1,1,1
24,3 | Where does Emma think the marble is?,0.000000,1,0,0


In [ ]:
df.assign(story_question=df["story_id"].astype(str) + " | " + df["question"]) \
  .groupby("story_question").is_correct.mean().sort_values(ascending=True).head(10)

story_question
8 | Where does Sally think Emma thinks the marble is?    0.0
9 | Where does Sally think Anne thinks the marble is?    0.0
5 | Where does Sally think the marble is?                0.0
5 | Where does Sally think Anne thinks the marble is?    0.0
9 | Where does Anne think Emma thinks the marble is?     0.0
9 | Where does Sally think Emma thinks the marble is?    0.0
1 | Where does Anne think the marble is?                 0.0
5 | Where does Anne think Sally thinks the marble is?    0.0
2 | Where does Emma think the marble is?                 0.0
3 | Where does Emma think the marble is?                 0.0
Name: is_correct, dtype: float64

In [179]:
df.groupby(["model", "qa_order"]).is_correct.mean()  # long story (steps=40) w/o peek/distraction, non-reasoning models

model                        qa_order
anthropic/claude-opus-4.1    1           0.70000
                             2           0.28125
anthropic/claude-sonnet-4    1           0.50000
                             2           0.25000
anthropic/claude-sonnet-4.5  1           0.60000
                             2           0.37500
Name: is_correct, dtype: float64

In [ ]:
df.groupby(["model", "qa_order"]).is_correct.mean()  # long story (steps=30), reasoning models except claude-sonnet-4.5

model                        qa_order
anthropic/claude-sonnet-4.5  1           0.666667
                             2           0.291667
google/gemini-2.5-pro        1           1.000000
                             2           0.937500
openai/gpt-5                 1           1.000000
                             2           0.916667
Name: is_correct, dtype: float64

In [ ]:
df.groupby(["model", "qa_order"]).is_correct.mean()  # reasoning models

model                        qa_order
anthropic/claude-opus-4.1    1           0.923077
                             2           0.794872
anthropic/claude-sonnet-4.5  1           1.000000
                             2           0.923077
openai/gpt-5                 1           1.000000
                             2           0.974359
Name: is_correct, dtype: float64

In [155]:
story_id = 3
row = df[df['story_id'] == story_id].head(1)
print(row['story_text'].iloc[0]); row['events'].iloc[0]
df[(df['story_id'] == story_id) & (df['question'] == 'Where does Anne think Emma thinks the marble is?')]

Emma entered the room.
Anne entered the room.
Emma put the marble into the basket.
Anne moved the marble to the box.
Sally entered the room.
Emma exited the room.
Anne exited the room.
Sally exited the room.


[MoveEvent(observers=Set({Observer(name='Emma', actual=True, perceived_by_others=True), Observer(name='Anne', actual=True, perceived_by_others=True)}), room='room', agent='Emma', object='marble', container='basket'),
 MoveEvent(observers=Set({Observer(name='Emma', actual=True, perceived_by_others=True), Observer(name='Anne', actual=True, perceived_by_others=True)}), room='room', agent='Anne', object='marble', container='box')]

,seed,story_id,story_text,model,story_steps,n_events,events,n_peeks,n_distractions,qa_order,qa_object,question,expected,answer,is_correct,latency_sec,retries
22,103,3,Emma entered the room.\nAnne entered the room....,anthropic/claude-sonnet-4,10,2,[MoveEvent(observers=Set({Observer(name='Emma'...,0,0,2,marble,Where does Anne think Emma thinks the marble is?,box,In the basket.,False,2.516307,0
82,103,3,Emma entered the room.\nAnne entered the room....,anthropic/claude-sonnet-4.5,10,2,[MoveEvent(observers=Set({Observer(name='Emma'...,0,0,2,marble,Where does Anne think Emma thinks the marble is?,box,basket,False,2.955589,0
157,103,3,Emma entered the room.\nAnne entered the room....,anthropic/claude-opus-4.1,10,2,[MoveEvent(observers=Set({Observer(name='Emma'...,0,0,2,marble,Where does Anne think Emma thinks the marble is?,box,In the basket,False,6.226905,0


In [100]:
# df.groupby(["model", "qa_order"]).is_correct.mean()
# df.groupby(["n_peeks", "n_distractions"]).is_correct.mean()
df.assign(story_question=df["story_id"].astype(str) + " | " + df["question"]) \
  .groupby("story_question").is_correct.mean().sort_values(ascending=True).head(10)

story_question
4 | Where does Sally think the marble is?                0.00
4 | Where does Sally think Emma thinks the marble is?    0.00
4 | Where does Sally think Anne thinks the marble is?    0.00
6 | Where does Sally think Emma thinks the marble is?    0.25
8 | Where does Emma think Sally thinks the marble is?    0.25
1 | Where does Anne think Emma thinks the marble is?     0.25
4 | Where does Emma think Anne thinks the marble is?     0.25
7 | Where does Sally think the marble is?                0.25
9 | Where does Emma think the marble is?                 0.50
6 | Where does Sally think Anne thinks the marble is?    0.50
Name: is_correct, dtype: float64

In [113]:
print(df[df['story_id'] == 4].head(1)['story_text'].iloc[0])

Emma entered the room.
Anne entered the room.
Emma put the marble into the basket.
As Anne, unnoticed by anyone, briefly got distracted and missed what happened, Emma moved the marble to the box.
Anne exited the room.
Emma exited the room.
Sally entered the room.


In [121]:
df[(df['story_id'] == 4) & (df['question'] == 'Where does Sally think the marble is?')]

,story_id,story_text,model,story_steps,n_events,n_peeks,n_distractions,qa_order,qa_object,question,expected,answer,is_correct,latency_sec,retries
59,4,Emma entered the room.\nAnne entered the room....,anthropic/claude-sonnet-4,10,2,1,2,1,marble,Where does Sally think the marble is?,basket,Unknown,False,12.343179,0
80,4,Emma entered the room.\nAnne entered the room....,anthropic/claude-sonnet-4.5,10,2,1,2,1,marble,Where does Sally think the marble is?,basket,Unknown,False,1.927029,0
139,4,Emma entered the room.\nAnne entered the room....,anthropic/claude-opus-4.1,10,2,1,2,1,marble,Where does Sally think the marble is?,basket,Unknown,False,2.619453,0
203,4,Emma entered the room.\nAnne entered the room....,openai/gpt-5,10,2,1,2,1,marble,Where does Sally think the marble is?,basket,Unknown,False,9.037114,0


In [84]:
engine.n_peeks, engine.n_distractions
print(story)
for k, v in results.items():
    print(k)
    v

(1, 1)

Emma entered the garden.
Anne entered the garden.
Anne put the marble into the basket.
Anne exited the garden.
Sally entered the garden.
Sally moved the marble to the box.
Sally exited the garden.
Emma moved the marble to the basket.
Emma exited the garden.
anthropic/claude-sonnet-4


[ModelResponse(model='anthropic/claude-sonnet-4', question='Where does Anne think the marble is?', expected='basket', answer='In the basket', is_correct=True, order=1, obj='marble', latency_sec=3.317863568663597, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4', question='Where does Emma think the marble is?', expected='basket', answer='In the basket', is_correct=True, order=1, obj='marble', latency_sec=2.8163426965475082, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4', question='Where does Sally think the marble is?', expected='box', answer='In the box.', is_correct=True, order=1, obj='marble', latency_sec=2.9623395614326, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4', question='Where does Anne think Emma thinks the marble is?', expected='basket', answer='In the basket.', is_correct=True, order=2, obj='marble', latency_sec=3.3918704725801945, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4', question='Where does Emma think Sally thi

anthropic/claude-sonnet-4.5


[ModelResponse(model='anthropic/claude-sonnet-4.5', question='Where does Anne think the marble is?', expected='basket', answer='basket', is_correct=True, order=1, obj='marble', latency_sec=2.584512896835804, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4.5', question='Where does Emma think the marble is?', expected='basket', answer='basket', is_correct=True, order=1, obj='marble', latency_sec=2.8413404896855354, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4.5', question='Where does Sally think the marble is?', expected='box', answer='box', is_correct=True, order=1, obj='marble', latency_sec=2.7293930016458035, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4.5', question='Where does Anne think Emma thinks the marble is?', expected='basket', answer='basket', is_correct=True, order=2, obj='marble', latency_sec=2.293307427316904, retries=0),
 ModelResponse(model='anthropic/claude-sonnet-4.5', question='Where does Emma think Sally thinks the marble is?

openai/gpt-5


[ModelResponse(model='openai/gpt-5', question='Where does Anne think the marble is?', expected='basket', answer='basket', is_correct=True, order=1, obj='marble', latency_sec=4.2958441413939, retries=0),
 ModelResponse(model='openai/gpt-5', question='Where does Emma think the marble is?', expected='basket', answer='the basket', is_correct=True, order=1, obj='marble', latency_sec=6.205942522734404, retries=0),
 ModelResponse(model='openai/gpt-5', question='Where does Sally think the marble is?', expected='box', answer='the box', is_correct=True, order=1, obj='marble', latency_sec=6.636925030499697, retries=0),
 ModelResponse(model='openai/gpt-5', question='Where does Anne think Emma thinks the marble is?', expected='basket', answer='basket', is_correct=True, order=2, obj='marble', latency_sec=9.769223533570766, retries=0),
 ModelResponse(model='openai/gpt-5', question='Where does Emma think Sally thinks the marble is?', expected='box', answer='the box', is_correct=True, order=2, obj='mar

In [177]:
eng = Engine(agents=agents, rooms=rooms,
            peek_prob=0., distracted_prob=0., exit_without_move_prob=0.2)
print(eng.generate_story(steps=40))
QAs = eng.generate_QAs(); QAs


break at step 39/40
Benjamin entered the room.
Sally entered the room.
Emma entered the room.
Aiden entered the room.
Sally put the marble into the cabinet.
Emma put the ball into the basket.
Benjamin moved the marble to the basket.
Isabella entered the room.
Aiden moved the marble to the box.
Aiden moved the marble to the cabinet.
Isabella moved the marble to the basket.
Emma moved the ball to the cabinet.
Emma moved the ball to the basket.
Sally moved the marble to the box.
Sally moved the marble to the cabinet.
Isabella moved the marble to the box.
Emma moved the marble to the basket.
Benjamin moved the marble to the box.
Sally moved the marble to the cabinet.
Sally exited the room.
Emma moved the ball to the cabinet.
Aiden moved the marble to the box.
Isabella moved the marble to the basket.
Emma moved the ball to the basket.
Aiden moved the marble to the box.
Isabella exited the room.
Benjamin moved the ball to the box.
Emma moved the ball to the basket.
Benjamin moved the ball t

{1: defaultdict(list,
             {'ball': [('Where does Sally think the ball is?', 'basket'),
               ('Where does Anne think the ball is?', 'cabinet'),
               ('Where does Emma think the ball is?', 'cabinet'),
               ('Where does Benjamin think the ball is?', 'basket'),
               ('Where does Aiden think the ball is?', 'basket'),
               ('Where does Isabella think the ball is?', 'basket')],
              'marble': [('Where does Sally think the marble is?', 'cabinet'),
               ('Where does Emma think the marble is?', 'cabinet'),
               ('Where does Benjamin think the marble is?', 'cabinet'),
               ('Where does Aiden think the marble is?', 'basket'),
               ('Where does Isabella think the marble is?', 'box')]}),
 2: defaultdict(list,
             {'ball': [('Where does Sally think Emma thinks the ball is?',
                'basket'),
               ('Where does Sally think Benjamin thinks the ball is?',
              

In [ ]:
events = []
observers = Set()
# Line 1: Emma, Charlotte, Benjamin, Aiden and Isabella entered the room.
observers.add(Observer('Emma'))
observers.add(Observer('Charlotte'))
observers.add(Observer('Benjamin'))
observers.add(Observer('Aiden'))
observers.add(Observer('Isabella'))
# Line 2: The pear is in the red treasure chest.
events.append(MoveEvent(observers=deepcopy(observers), agent='System', object='pear', container='red treasure chest'))
# Line 3: Emma moved the pear to the blue suitcase.
events.append(MoveEvent(observers=deepcopy(observers), agent='Emma', object='pear', container='blue suitcase'))
# Line 4: Charlotte exited the room.
observers.remove_by_name('Charlotte')
# Line 5: Benjamin lost his watch.
events.append(OtherEvent(observers=deepcopy(observers), agent='Benjamin', object='his watch', verb='lost'))
# Other event: Benjamin lost his watch
# Line 6: Benjamin exited the room.
observers.remove_by_name('Benjamin')
# Line 7: Aiden moved the pear to the blue crate.
events.append(MoveEvent(observers=deepcopy(observers), agent='Aiden', object='pear', container='blue crate'))
# Line 8: Emma exited the room.
observers.remove_by_name('Emma')
# Line 9: Aiden exited the room.
observers.remove_by_name('Aiden')
# Line 10: Isabella moved the pear to the red treasure chest.
events.append(MoveEvent(observers=deepcopy(observers), agent='Isabella', object='pear', container='red treasure chest'))
# Line 11: Isabella likes the red box.
events.append(OtherEvent(observers=deepcopy(observers), agent='Isabella', object='the red box', verb='likes'))
# Other event: Isabella likes the red box
# Line 12: Isabella exited the room.
observers.remove_by_name('Isabella')
# Question: Where does Emma think the pear is?
safe_get([event for event in events if event.observers.has('Emma', actual=True) and event.object == 'pear'], -1, 'container')

'blue crate'

In [ ]:
# THEORY OF MIND QUESTION ANSWERING

def answer_tom_questions(events, object_name='pear'):
    """Answer common Theory of Mind questions based on parsed events"""
    
    print("=== THEORY OF MIND ANALYSIS ===")
    
    # Get all unique observers who were ever present
    all_observers = set()
    for event in events:
        all_observers.update(obs.name for obs in event.observers)
    
    print(f"Object being tracked: {object_name}")
    print(f"All observers in story: {list(all_observers)}")
    print()
    
    # Question 1: Where is the object really?
    real_location = safe_get([event for event in events if event.object == object_name], 
                           -1, 'container', 'Unknown')
    print(f"1. Where is the {object_name} really? → {real_location}")
    
    # Question 2-N: Where does each observer think the object is?
    for observer in sorted(all_observers):
        belief = safe_get([event for event in events 
                          if event.observers.has(observer, actual=True) and event.object == object_name], 
                         -1, 'container', 'Unknown')
        print(f"2. Where does {observer} think the {object_name} is? → {belief}")
    
    print()
    
    # Question 3: Second-order beliefs (X thinks Y thinks...)
    print("=== SECOND-ORDER BELIEFS ===")
    observer_list = sorted(all_observers)
    for i, obs1 in enumerate(observer_list):
        for j, obs2 in enumerate(observer_list):
            if i != j:  # Don't ask "X thinks X thinks..."
                # Find last event where obs1 was actually present AND obs2 was perceived by others
                shared_belief = safe_get([event for event in events 
                                        if event.observers.has(obs1, actual=True) and 
                                           event.observers.has(obs2, perceived_by_others=True) and 
                                           event.object == object_name], 
                                       -1, 'container', 'Unknown')
                print(f"3. Where does {obs1} think {obs2} thinks the {object_name} is? → {shared_belief}")
    
    return {
        'real_location': real_location,
        'first_order_beliefs': {obs: safe_get([event for event in events 
                                             if event.observers.has(obs, actual=True) and event.object == object_name], 
                                            -1, 'container', 'Unknown') 
                               for obs in all_observers}
    }

# Answer questions about the parsed story
results = answer_tom_questions(events, 'pear')


In [ ]:
d = json.load(open('Hi-ToM_data.json'))['data']
df = pd.DataFrame(d)
df.head()

In [10]:
df['question_order'].value_counts().sort_index()

0    240
1    240
2    240
3    240
4    240
Name: question_order, dtype: int64

In [261]:
# Show full contents only for this cell (using context manager)
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None, 
                       'display.width', None, 
                       'display.max_colwidth', None):
    # Display all question_order and story data
    print(sample[['question_order', 'question']])

     question_order  \
237               1   
276               3   
633               1   
158               2   
423               1   

                                                              question  
237                     Where does Elizabeth really think the lime is?  
276  Where does Jack think Jackson thinks Emma thinks the eggplant is?  
633                     Where does Mila really think the tangerine is?  
158              Where does Nathan think Jackson thinks the tomato is?  
423                    Where does Isabella really think the potato is?  


In [11]:
sample = df[df['question_order'] > 0].sample(5)

In [13]:
sample[['question_order', 'story']]

,question_order,story
237,1,"1 Elizabeth, Alexander, Nathan, Owen and Jacks..."
276,3,"1 Emily, Avery, Emma, Jack and Jackson entered..."
633,1,"1 Mila, Chloe, Amelia, Benjamin and Elizabeth ..."
158,2,"1 Sophia, Jackson, Amelia, Charlotte and Natha..."
423,1,Read the following story and answer the multip...


In [262]:
for q_order, story, question in zip(sample['question_order'], sample['story'], sample['question']):
    print(f"\n--- Question order {q_order} ---")
    print(story)
    print("-" * 20)
    print(question)
    print("-" * 50)


--- Question order 1 ---
1 Elizabeth, Alexander, Nathan, Owen and Jackson entered the cellar.
2 Jackson lost his watch.
3 The lime is in the blue_crate.
4 Elizabeth moved the lime to the green_pantry.
5 Elizabeth exited the cellar.
6 Nathan saw a monkey.
7 Alexander made no movements and stayed in the cellar for 1 minute.
8 Alexander exited the cellar.
9 Nathan moved the lime to the green_basket.
10 Nathan exited the cellar.
11 Owen moved the lime to the red_box.
12 Owen exited the cellar.
13 Jackson moved the lime to the blue_crate.
14 Jackson exited the cellar.
15 Elizabeth, Alexander, Nathan, Owen and Jackson entered the waiting_room.
16 Nathan, Jackson and Elizabeth entered the workshop.
17 The sweet_potato is in the blue_suitcase.
18 Nathan made no movements and stayed in the workshop for 1 minute.
19 Nathan exited the workshop.
20 Jackson made no movements and stayed in the workshop for 1 minute.
21 Jackson exited the workshop.
22 Elizabeth made no movements and stayed in the wo

In [ ]:
story = '''Emma, Charlotte, Benjamin, Aiden and Isabella entered the room.
The pear is in the red treasure chest.
Emma moved the pear to the blue suitcase.
Charlotte exited the room.
Benjamin lost his watch.
Benjamin exited the room.
Aiden moved the pear to the blue crate.
Emma exited the room.
Aiden exited the room.
Isabella moved the pear to the red treasure chest.
Isabella likes the red box.
Isabella exited the room.'''

# story in code
events: List[Event] = []
observers = Set()
observers.add(Observer('Sally'))  # Sally entered
observers.add(Observer('Anne'))   # Anne entered
observers.add(Observer('Emma'))   # Emma entered
# Sally moved the marble to the basket
events.append(MoveEvent(observers=deepcopy(observers), agent='Sally', object='marble', container='basket'))
observers.remove_by_name('Sally') # Sally left
# Anne moved the marble to the box. During this action,
# Emma was distracted and did not notice what happened,
# and Sally peeked from outside.
events.append(MoveEvent(observers=deepcopy(observers)
                        .update_by_name('Emma', actual=False) # Emma was distracted
                        .add(Observer('Sally', perceived_by_others=False)), # Sally peeked
                        agent='Anne', object='marble', container='box'))
# events.append(MoveEvent(observers=deepcopy(observers), agent='Anne', object='marble', container='box'))
# events[-1].observers.update_by_name('Emma', actual=False)  # Emma was distracted
# events[-1].observers.add(Observer('Sally', perceived_by_others=False))  # Sally peeked
observers.remove_by_name('Anne')  # Anne left

# questions
# Where is the marble? Box
safe_get([event for event in events if event.object == 'marble'], index=-1, field='container')

# Where does Sally think the marble is? Box
safe_get([event for event in events if event.observers.has('Sally', actual=True) and 
          event.object == 'marble'], index=-1, field='container')

# Where does Sally think Anne thinks the marble is? Box
safe_get([event for event in events if event.observers.has('Sally', actual=True) and 
          event.observers.has('Anne', perceived_by_others=True) and 
          event.object == 'marble'], index=-1, field='container')

# Where does Anne think Sally thinks the marble is? Basket
safe_get([event for event in events if event.observers.has('Anne', actual=True) and 
          event.observers.has('Sally', perceived_by_others=True) and 
          event.object == 'marble'], index=-1, field='container')

# Where does Emma think the marble is? Basket
safe_get([event for event in events if event.observers.has('Emma', actual=True) and 
          event.object == 'marble'], index=-1, field='container')

# Where does Anne think Emma thinks the marble is? Box
safe_get([event for event in events if event.observers.has('Anne', actual=True) and 
          event.observers.has('Emma', perceived_by_others=True) and 
          event.object == 'marble'], index=-1, field='container')

{Observer(name='Sally', actual=True, perceived_by_others=True)}

{Observer(name='Anne', actual=True, perceived_by_others=True),
 Observer(name='Sally', actual=True, perceived_by_others=True)}

{Observer(name='Anne', actual=True, perceived_by_others=True),
 Observer(name='Emma', actual=True, perceived_by_others=True),
 Observer(name='Sally', actual=True, perceived_by_others=True)}

{Observer(name='Anne', actual=True, perceived_by_others=True),
 Observer(name='Emma', actual=True, perceived_by_others=True)}

{Observer(name='Emma', actual=True, perceived_by_others=True)}

...

'box'

'box'

'basket'

'basket'

'box'